# Setup

In [2]:
pip install openai==1.12.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 967.9 kB/s eta 0:00:00


In [3]:
import openai
from openai import OpenAI

print(openai.__version__) # version - 1.12.0


1.12.0


In [4]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)


Mounted at /content/drive/


In [ ]:
pwd

'/content'

In [5]:
cd /content/drive/MyDrive/

/content/drive/MyDrive


In [6]:
cd 'Colab Notebooks'/'Designing Smart and Healthy Systems - Spring 2024'/HealthLLM/LLM_MVP

/content/drive/.shortcut-targets-by-id/1PMflFOZrgZRP9PQZZLuq7i16kKifkjSi/Designing Smart and Healthy Systems - Spring 2024/HealthLLM/LLM_MVP


In [ ]:
ls

credentials.json  data_2.json  data_3.json  data.json  starter_code.ipynb


### storing real credentials.json inside a google drive folder is not secure and not best practice
### please download the folder, create a github repo, then create a .gitignore

In [7]:
ls

credentials.json  data_2.json  data_3.json  data.json  starter_code.ipynb


# Load data and credentials

In [8]:
import json

# Update the path if your file is in a different location
file_path = 'data.json'

with open(file_path, 'r') as file:
    patient_data = json.load(file)

# To verify, print a part of the data
print(patient_data['patient_demographics'])


{'name': 'John Doe', 'age': 70, 'gender': 'Male', 'admission_date': '2024-02-10', 'discharge_date': '2024-02-14'}


In [9]:
# You should not store API keys in a google drive folder - it is not secure. If using github, add credentials.json to .gitignore.

credentials_path = '/content/credentials.json'

with open(credentials_path, 'r') as file:
    credentials = json.load(file)

openai_api_key = credentials['openai_api_key']

# Use the API key as needed, for example, to configure OpenAI's API client
openai.api_key = openai_api_key


# Explore data

In [10]:
print(f'Data type: {type(patient_data)}')

Data type: <class 'dict'>


In [11]:
print(f'Keys in data: {(patient_data.keys())}')

Keys in data: dict_keys(['patient_id', 'patient_demographics', 'diagnoses', 'drg', 'encounters', 'flowsheets', 'imaging', 'labs', 'med_orders', 'notes'])


In [12]:
# Look at encounters data, there are multiple
patient_data['encounters']

[{'date': '2024-02-10',
  'type': 'Admission',
  'reason': 'Cough, shortness of breath, hemoptysis, fever'},
 {'date': '2024-02-14', 'type': 'Discharge'}]

In [13]:
# Look at flowsheets data, there are multiple
patient_data['flowsheets']

[{'date': '2024-02-10',
  'time': '08:00',
  'temperature': '38.5°C',
  'heart_rate': '90 bpm',
  'blood_pressure': '130/85 mmHg',
  'respiratory_rate': '20 breaths/min',
  'oxygen_saturation': '92%'},
 {'date': '2024-02-11',
  'time': '08:00',
  'temperature': '37.8°C',
  'heart_rate': '85 bpm',
  'blood_pressure': '125/80 mmHg',
  'respiratory_rate': '18 breaths/min',
  'oxygen_saturation': '94%'},
 {'date': '2024-02-12',
  'time': '08:00',
  'temperature': '37°C',
  'heart_rate': '80 bpm',
  'blood_pressure': '120/75 mmHg',
  'respiratory_rate': '16 breaths/min',
  'oxygen_saturation': '96%'},
 {'date': '2024-02-13',
  'time': '08:00',
  'temperature': '36.5°C',
  'heart_rate': '78 bpm',
  'blood_pressure': '118/74 mmHg',
  'respiratory_rate': '14 breaths/min',
  'oxygen_saturation': '98%'}]

In [14]:
patient_data['notes']

[{'date': '2024-02-10',
  'author': 'Dr. Smith',
  'note_type': 'Admission Note',
  'content': 'Patient admitted with symptoms indicative of pneumonia. Initiated empirical antibiotic therapy.'},
 {'date': '2024-02-11',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': 'Patient shows signs of improvement. CRP levels declining. Continue with current antibiotic regimen. Monitor for any adverse reactions.'},
 {'date': '2024-02-12',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': "Patient's temperature normalized. Oxygen saturation levels stable. Plan to switch to oral antibiotics tomorrow."},
 {'date': '2024-02-13',
  'author': 'Dr. Smith',
  'note_type': 'Ward Round Note',
  'content': 'Patient continues to improve. CRP and WBC trending down. Ready for discharge planning.'},
 {'date': '2024-02-14',
  'author': 'Dr. Smith',
  'note_type': 'Discharge Summary',
  'content': "Patient's condition improved with treatment. Medically fit for discharge

# Sample zero-shot solution
This is a first attempt where we pass in everything and provide basic prompt engineering. This is a baseline solution. Your solutions should improve from this.

In [15]:
client = OpenAI(api_key=openai_api_key)

In [19]:
def generate_patient_summary(patient_data, additional_prompts='', instructions=''):
    """
    Generates a patient summary using OpenAI's GPT-3.5.

    Parameters:
    - patient_data (str): The patient data to generate the summary from.
    - additional_prompts (str): Any additional prompts

    Returns:
    - str: The generated patient summary.
    """

    # Initialize the OpenAI client with your API key
    client = OpenAI(api_key=openai_api_key)

    # Create the chat completion request with the patient data
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"""
                              You are a doctor writing a discharge letter for a patient. Use patient data from context only. Minimize hallucinations.
                              {additional_prompts}
                              Write the output in this format {instructions}
                              Data: {patient_data}
                """,
            }
        ],
        model="gpt-3.5-turbo",
    )

    # Return the content of the generated message
    return chat_completion.choices[0].message.content

In [18]:
#Instructions for a good discharge summary :

# Define the path to your file
file_path = '/content/How to write a discharge summary.txt'

# Open the file using 'open()' in read mode ('r')
with open(file_path, 'r') as file:
    # Read the contents of the file
    instructions = file.read()

    # Print the contents
    print(instructions)

How to write a discharge summary

General Principle:
A discharge summary should not be too lengthy. I should however, be informative to the outpatient provider and during future admissions. 

Outstanding issues
•	Labs that need to be ordered / followed up on, either by the primary care provider or a the Nursing Home:
o	The patient should have a Hct and Cr rechecked in 2 weeks
•	Provide Instructions to the Visiting Nurse; be specific:
o	Please weigh patient every day. Call MD at xxx-xxx-xxxx if 5 lbs over y weight. 
•	List appointments that need to be scheduled; Give MD / subspecialty and contact info. 

HPI
•	Do not copy from H&P. Instead, write a small paragraph that summarizes the reason for admission. 

PMHx (can copy forward from H&P if you have already put this information in previously)
•	Give the pertinent information along with concurrent diagnoses:
o	CHF (EF 30% in 4/05)
o	CAD with h/o 3 MI’s in 1991, 1995, 2000
o	CAD (70% LAD lesion diagnosed by cath at Brigham in 4/07)
o	HIV

In [20]:
print(generate_patient_summary(patient_data,'Make sure that you execute this piecewise in the format based on the instuctions for output format ',instructions))

**How to write a discharge summary**

**Outstanding issues:**
- Labs that need to be ordered / followed up on: The patient should have a Hct and Cr rechecked in 2 weeks.
- Provide Instructions to the Visiting Nurse: Please weigh the patient every day. Call MD at xxx-xxx-xxxx if 5 lbs over y weight.

**HPI:**
- Mr. John Doe, a 70-year-old male, was admitted on 10th February 2024 with symptoms of cough, shortness of breath, hemoptysis, and fever.

**PMHx:**
- CHF (EF 30% in 4/05)
- CAD with h/o 3 MI’s in 1991, 1995, 2000
- CAD (70% LAD lesion diagnosed by cath at Brigham in 4/07)
- HIV (CD4 317, VL < 75 in 5/07)

**Meds:** 
- Amoxicillin IV 500 mg 8 hourly
- Paracetamol PRN
- Atorvastatin PO 20 mg Nightly

**Hospital Course:**
- In the ED, the patient presented with a temperature of 38.5°C, CRP 60 mg/L, and consolidation in the left lower lobe on chest X-ray.
- On the floor, the patient received IV Amoxicillin and showed improvement with decreasing CRP levels.

**Discharge Condition:**
-